In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import glob
import re
import itertools
import time
import joblib
import random

import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = 15
pd.options.display.max_colwidth = -1
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import plot_precision_recall_curve
from sklearn.metrics import average_precision_score, auc, roc_curve
from sklearn import svm, ensemble, datasets

In [3]:
import parsl
parsl.clear()

from polyfuse.configs.local import config
parsl.load(config)

#parsl.load()

from polyfuse import apps, transformations

In [4]:
out_dir = '/cephfs/users/annawoodard/polyfuse/data/sim_50/processed'
training_fraction = 0.75

In [5]:
truth = apps.concatenate_true_fusions('/cephfs/users/annawoodard/fusion-simulation/data/processed/*', out_dir)

In [6]:
truth = pd.read_hdf(truth.result(), 'data')

In [7]:
callers = ['starseqr', 'starfusion', 'arriba', 'fusioncatcher', 'pizzly']
indices = dict((c, i) for i, c in enumerate(callers))

In [8]:
parsed_caller_data = apps.parse_caller_data(out_dir, callers)

In [9]:
caller_data_path = apps.concatenate_caller_data(out_dir, inputs=parsed_caller_data)
caller_data = pd.read_hdf(caller_data_path.result(), 'data')

In [10]:
samples = sorted(caller_data['sample'].unique())
#random.shuffle(samples)
training_samples = samples[:int(len(samples) * training_fraction)]
testing_samples = samples[int(len(samples) * training_fraction):]

In [11]:
data = [apps.assemble_data(sample, callers, out_dir) for sample in samples]

start = time.time()
X = np.array(sum([data.result()[0] for data in data], []))
Y = np.array(sum([data.result()[1] for data in data], []))

print('assembled data in {:.1f}s'.format((time.time() - start)))

assembled data in 37.8s


In [12]:
X.shape
#import sklearn
#sorted(sklearn.metrics.SCORERS.keys())

(71571, 5)

In [ ]:
start = time.time()

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

n = 12000
X_train, X_test, Y_train, Y_test = train_test_split(
    X[n:], Y[n:], test_size=0.5, random_state=0)


param_grid = [
    {'kernel': ['rbf'],
     'gamma': [1e-3, 1e-4],
     'C': [1, 10, 100, 1000]
    },
    {'kernel': ['linear'],
     'C': [1, 10, 100, 1000]
    }
]


param_grid = {
    'kernel': ['rbf'],
    'C': [1, 10, 100, 1000],
    'gamma': [1e-3, 1e-4, 1e-6, 1e-7, 'auto']
}

clf = GridSearchCV(
    svm.SVC(), param_grid, scoring='roc_auc', n_jobs=-1, cv=3
)
clf.fit(transformations.flatten(X_train), Y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

print('tuned parameters in {:.1f}s'.format((time.time() - start)))

In [ ]:
training_data = [apps.assemble_data(sample, callers, out_dir) for sample in training_samples]

start = time.time()
X_train = np.array(sum([data.result()[0] for data in training_data], []))
Y_train = np.array(sum([data.result()[1] for data in training_data], []))

print('assembled data in {:.1f}s'.format((time.time() - start)))

In [ ]:
1 / (5 * X_train.var())

In [ ]:
start = time.time()
os.makedirs(os.path.join(out_dir, 'models'), exist_ok=True)
classifiers = []
for features in [
            #['pizzly', 'starseqr', 'fusioncatcher'],
            ['pizzly', 'starfusion', 'arriba', 'fusioncatcher'],
            callers,
            #['arriba']
            #['STARSEQR', 'PIZZLY', 'PRADA', 'FUSIONCATCHER_v1.10_June192019', 'ChimeraScan', 'ARRIBA', 'MapSplice'],
            #['STARSEQR', 'PIZZLY', 'PRADA', 'FUSIONCATCHER_v1.10_June192019', 'ChimeraScan', 'ARRIBA', 'MapSplice', 'STAR_FUSION_v1.5'],
            #['STARSEQR', 'ARRIBA', 'STAR_FUSION_v1.5']
        ]:
    
    pruned_indices = [indices[f] for f in features]
    
    label = 'LSVC{}Features'.format(len(features))
    lsvc_trans = 'flatten'
    lsvc = svm.LinearSVC()
    print(label, lsvc.get_params())
    lsvc.fit(getattr(transformations, lsvc_trans)(X_train[:, pruned_indices]), Y_train)
    joblib.dump(lsvc, os.path.join(out_dir, 'models', label + '.joblib'))
    classifiers += [(label, pruned_indices, lsvc_trans)]

    label = 'DefaultSVC{}Features'.format(len(features))
    svc_trans = 'flatten'
    svc = svm.SVC()
    print(label, svc.get_params())
    svc.fit(getattr(transformations, svc_trans)(X_train[:, pruned_indices]), Y_train)
    joblib.dump(svc, os.path.join(out_dir, 'models', label + '.joblib'))
    classifiers += [(label, pruned_indices, svc_trans)]

    label = 'SVC{}Features'.format(len(features))
    svc_trans = 'flatten'
    svc = svm.SVC(C=1, gamma=0.0001)
    print(label, svc.get_params())
    svc.fit(getattr(transformations, svc_trans)(X_train[:, pruned_indices]), Y_train)
    joblib.dump(svc, os.path.join(out_dir, 'models', label + '.joblib'))
    classifiers += [(label, pruned_indices, svc_trans)]
    
    label = 'RFC{}Features'.format(len(features))
    rfc_trans = 'noop'
    rfc = ensemble.RandomForestClassifier()
    rfc.fit(getattr(transformations, rfc_trans)(X_train[:, pruned_indices]), Y_train)
    joblib.dump(rfc, os.path.join(out_dir, 'models', label + '.joblib'))
    classifiers += [(label, pruned_indices, rfc_trans)]
    
    label = 'GBC{}Features'.format(len(features))
    gbc_trans = 'noop'
    gbc = ensemble.GradientBoostingClassifier()
    gbc.fit(getattr(transformations, rfc_trans)(X_train[:, pruned_indices]), Y_train)
    joblib.dump(gbc, os.path.join(out_dir, 'models', label + '.joblib'))
    classifiers += [(label, pruned_indices, gbc_trans)]
    
print('fit models in {:.1f}s'.format((time.time() - start)))

In [ ]:
X_train.shape, sum(Y_train)

In [ ]:
testing_data = [apps.assemble_data(sample, callers, out_dir) for sample in testing_samples]

In [ ]:
start = time.time()
X_test = np.array(sum([data.result()[0] for data in testing_data], []))
Y_test = np.array(sum([data.result()[1] for data in testing_data], []))

print('assembled testing data in {:.1f}s'.format((time.time() - start)))

In [ ]:
X_test.shape, Y_test.shape

In [ ]:
start = time.time()
futures = []
for sample in testing_samples:
    for label, feature_indices, transformation in classifiers:
        futures += [apps.predict(sample, out_dir, label, feature_indices, transformation, callers)]
polyfuse_data = pd.concat([f.result() for f in futures])
print('assembled predictions in {:.1f}s'.format((time.time() - start)))

In [ ]:
polyfuse_data.to_hdf(os.path.join(out_dir, 'polyfuse_data.hdf'), 'data', mode='w')

In [ ]:
start = time.time()
truth_path = os.path.join(out_dir, 'true_fusions.hdf')
futures = [apps.score(sample, os.path.join(out_dir, 'polyfuse_data.hdf'), truth_path) for sample in testing_samples]
futures += [apps.score(sample, os.path.join(out_dir, 'caller_data.hdf'), truth_path) for sample in testing_samples]
summary = pd.concat([f.result() for f in futures])
print('assembled scores in {:.1f}s'.format((time.time() - start)))

In [ ]:
#ax = sns.boxplot(x="caller", y="average PR", data=summary)
ax = sns.boxplot(x="caller", y="auc", data=summary.sort_values(by='auc', ascending=False))
labels = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
_ = plt.setp(ax.artists, edgecolor = 'k', facecolor='w')
_ = plt.setp(ax.lines, color='k')
os.makedirs('plots', exist_ok=True)
plt.savefig('plots/auc_RFC_SVC_3-7-8-23_features_sum_J_S.pdf')

In [75]:
pd.options.display.max_rows = None
summary.groupby('caller').mean().sort_values(by='auc', ascending=False)

,auc,average PR,total FP,total FN,total TP
caller,,,,,
polyfuseRFC5Features,0.805714,0.939604,32.85,71.25,428.75
polyfuseRFC4Features,0.799550,0.939043,32.35,74.15,425.85
polyfuseGBC5Features,0.752892,0.914705,42.85,87.65,412.35
polyfuseGBC4Features,0.751038,0.909081,46.70,86.20,413.80
polyfuseLSVC4Features,0.725535,0.901864,47.65,96.30,403.70
polyfuseLSVC5Features,0.724395,0.909399,42.35,100.10,399.90
polyfuseSVC4Features,0.721631,0.912547,40.75,103.00,397.00
polyfuseSVC5Features,0.719811,0.903647,45.25,100.00,400.00
starfusion,0.714148,0.892422,54.45,98.60,401.40


In [63]:
summary.sort_values(by='auc', ascending=False)

,sample,caller,auc,average PR,total FP,total FN,total TP
0,LIB-04642wt,polyfuseGBC5Features,0.797993,0.940913,39,77,423
0,LIB-04655wt,polyfuseGBC5Features,0.787991,0.926455,42,75,425
0,LIB-04657wt,polyfuseGBC4Features,0.785548,0.929155,37,77,423
0,LIB-04653wt,polyfuseGBC5Features,0.783607,0.920847,33,73,427
0,LIB-04650wt,polyfuseGBC5Features,0.782698,0.921139,44,75,425
0,LIB-04655wt,polyfuseGBC4Features,0.782121,0.923471,45,77,423
0,LIB-04655wt,polyfuseLSVC4Features,0.780568,0.935511,36,83,417
0,LIB-04642wt,polyfuseGBC4Features,0.780477,0.933984,41,83,417
0,LIB-04657wt,polyfuseGBC5Features,0.778095,0.922230,42,78,422
0,LIB-04642wt,polyfuseRFC5Features,0.777375,0.914971,53,76,424
